# Quarto Projeto

O dataset utilizado representa um site de vendas de carros, onde cada linha representa um carro que está sendo vendido. Alguns desses carros foram vendidos, e outros não, essa informação é representada pela coluna "sold". Temos 3 features sobre esses carros, que podem refletir na venda ou nao venda do mesmo. Elas são representadas pelas colunas "mileage_per_year", "model_year" e "price"

#### Configurando o dataset

In [44]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/4d1d4a16ccbf6ea4e0a64a38a24ec884/raw/afd05cb0c796d18f3f5a6537053ded308ba94bf7/car-prices.csv"

dataset = pd.read_csv(uri)
dataset.head()


,Unnamed: 0,mileage_per_year,model_year,price,sold
0,0,21801,2000,30941.02,yes
1,1,7843,1998,40557.96,yes
2,2,7109,2006,89627.50,no
3,3,26823,2015,95276.14,no
4,4,7935,2014,117384.68,yes


Renomeando as colunas

In [45]:
rename_map = {
  "mileage_per_year": "milhas_por_ano",
  "model_year": "ano_do_modelo",
  "price": "preco",
  "sold": "vendido"
}

dataset = dataset.rename(columns=rename_map)
dataset.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido
0,0,21801,2000,30941.02,yes
1,1,7843,1998,40557.96,yes
2,2,7109,2006,89627.50,no
3,3,26823,2015,95276.14,no
4,4,7935,2014,117384.68,yes


Swaping da coluna "vendido"

In [46]:
swap = {
  "no": 0,
  "yes": 1
}

dataset["vendido"] = dataset.vendido.map(swap)
dataset.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido
0,0,21801,2000,30941.02,1
1,1,7843,1998,40557.96,1
2,2,7109,2006,89627.50,0
3,3,26823,2015,95276.14,0
4,4,7935,2014,117384.68,1


Criando a feature "idade_do_modelo"

In [47]:
from datetime import datetime

ano_atual = datetime.today().year
dataset["idade_do_modelo"] = ano_atual - dataset.ano_do_modelo 
dataset.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido,idade_do_modelo
0,0,21801,2000,30941.02,1,23
1,1,7843,1998,40557.96,1,25
2,2,7109,2006,89627.50,0,17
3,3,26823,2015,95276.14,0,8
4,4,7935,2014,117384.68,1,9


Criando a feature "km por ano"

In [48]:
# 1 milha equivale a 1.60934 km

dataset["km_por_ano"] = dataset.milhas_por_ano * 1.60934
dataset.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido,idade_do_modelo,km_por_ano
0,0,21801,2000,30941.02,1,23,35085.22134
1,1,7843,1998,40557.96,1,25,12622.05362
2,2,7109,2006,89627.50,0,17,11440.79806
3,3,26823,2015,95276.14,0,8,43167.32682
4,4,7935,2014,117384.68,1,9,12770.11290


Removendo colunas que não utilizaremos

In [49]:
useless_columns = ["Unnamed: 0", "milhas_por_ano", "ano_do_modelo"] 
dataset = dataset.drop(columns=useless_columns)
dataset.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,23,35085.22134
1,40557.96,1,25,12622.05362
2,89627.50,0,17,11440.79806
3,95276.14,0,8,43167.32682
4,117384.68,1,9,12770.11290


Separando features das labels

In [50]:
x = dataset[["preco", "idade_do_modelo", "km_por_ano"]]
y = dataset["vendido"]
x.head()

,preco,idade_do_modelo,km_por_ano
0,30941.02,23,35085.22134
1,40557.96,25,12622.05362
2,89627.50,17,11440.79806
3,95276.14,8,43167.32682
4,117384.68,9,12770.11290


#### Separando treino e teste

In [51]:
from sklearn.model_selection import train_test_split
import numpy as np

SEED = 5
np.random.seed(SEED)

train_x, test_x, train_y, test_y = train_test_split(x, y, stratify=y, test_size=0.25)
print(f"Treino com {len(train_x)} elementos e teste com {len(test_x)}")

Treino com 7500 elementos e teste com 2500


#### Pré-processamento

Normalizando os dados com scaler

In [52]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

raw_train_x = train_x
raw_test_x = test_x

scaler.fit(raw_train_x)

train_x = scaler.transform(raw_train_x)
test_x = scaler.transform(raw_test_x)

#### Importando o modelo

In [53]:
from sklearn.svm import SVC

model = SVC(gamma="auto")

#### Treinando o modelo

In [54]:
model.fit(train_x, train_y)

SVC(gamma='auto')

#### Validando o modelo

In [55]:
from sklearn.metrics import accuracy_score

predictions = model.predict(test_x)
accuracy = accuracy_score(test_y, predictions) * 100

print(f"Accuracy of model in {len(test_y)} elements of data test was {accuracy:.2f}%")

Accuracy of model in 2500 elements of data test was 77.48%
